In [1]:
from fyers_api import accessToken
from fyers_api.Websocket import ws
import pandas as pd
from fyers_api import fyersModel
import os
from fyers_api.Websocket import ws
import numpy as np
import math
import datetime

In [2]:
client_id='TAZ287TZUA-100'
secret_id='PPZ5ZG1YAR'
url='https://www.google.com/'
response_type = "code"
grant_type = "authorization_code"

In [3]:
session=accessToken.SessionModel(
    client_id=client_id,
    secret_key=secret_id,
    redirect_uri=url, 
    response_type=response_type,
    grant_type=grant_type
)

response = session.generate_authcode()
print(response)

https://api.fyers.in/api/v2/generate-authcode?client_id=TAZ287TZUA-100&redirect_uri=https%3A%2F%2Fwww.google.com%2F&response_type=code&state=None


In [4]:
auth_code='eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE2OTYwNjA2ODYsImV4cCI6MTY5NjA5MDY4NiwibmJmIjoxNjk2MDYwMDg2LCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJYTTA0MzQyIiwib21zIjoiSzEiLCJoc21fa2V5IjoiNWI3Mzg3ZTYxOGQzODllNGVjOGVmZjU1ODM0ZTBmMTA1NjdjYWU1ODk5YWQ5NWY3MzgxMTM3YTMiLCJub25jZSI6IiIsImFwcF9pZCI6IlRBWjI4N1RaVUEiLCJ1dWlkIjoiN2U1NmE1YTlhNGI1NDg4ZmI5YTdkMmJlNjk1NTAyOTAiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.nk-V9aFA58SkD5wY0IDWlsV5by0o4iHDBe1gHpiXilg'

In [5]:
session.set_token(auth_code)
response = session.generate_token()
print(response)

{'s': 'ok', 'code': 200, 'message': '', 'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE2OTYwNjA3MDUsImV4cCI6MTY5NjEyMDIyNSwibmJmIjoxNjk2MDYwNzA1LCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbEY5VWhhckFEX0hQREh3cU5iOURpVWdDNUNCNm8yWVljVWk5WmwwaVpWNXBLX1lHd1pjU3dfSHZTcWRVNUw0VVk1OE1nNHduN1d0M2FpOUEwUXBVUF9FMzJKQXhYWDVmU3F5ekRVYl8zQm9CNDhubz0iLCJkaXNwbGF5X25hbWUiOiJNT0hBTU1FRCBSQVlZQU4gU0FJVCIsIm9tcyI6IksxIiwiaHNtX2tleSI6IjViNzM4N2U2MThkMzg5ZTRlYzhlZmY1NTgzNGUwZjEwNTY3Y2FlNTg5OWFkOTVmNzM4MTEzN2EzIiwiZnlfaWQiOiJYTTA0MzQyIiwiYXBwVHlwZSI6MTAwLCJwb2FfZmxhZyI6Ik4ifQ.ezffwTYpNMdgqKxkUYDCUgAxIgeCKG0bMrNakUamT8g', 'refresh_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE2OTYwNjA3MDUsImV4cCI6MTY5NzMyOTgyNSwibmJmIjoxNjk2MDYwNzA1LCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6InJlZnJlc2hfdG9rZW4iLCJhdF9oYXNoIjoiZ0FBQU

In [6]:
access_token=response['access_token']

In [7]:
def sma(df: pd.DataFrame):
    small_lookback = 5
    df['mean'] = df['closingValues'].ewm(span=5).mean()
    window = 10  # You can adjust this window size as needed
    std_dev_multiplier = 1.5  # Multiplier for standard deviation

    # Calculate the rolling mean (SMA) and rolling standard deviation
    df['SMA'] = df['candleStick'].apply(lambda x: pd.Series(x[4])).rolling(window=window).mean()
    df['StdDev'] = df['candleStick'].apply(lambda x: pd.Series(x[4])).rolling(window=window).std()

    # Calculate the upper and lower Bollinger Bands
    df['Upper_Band'] = df['SMA'] + (std_dev_multiplier * df['StdDev'])
    df['Lower_Band'] = df['SMA'] - (std_dev_multiplier * df['StdDev'])



In [8]:



def rsi_tradingview(ohlc: pd.DataFrame, period: int = 14, round_rsi: bool = True):
    """ Implements the RSI indicator as defined by TradingView on March 15, 2021.
        The TradingView code is as follows:
        //@version=4
        study(title="Relative Strength Index", shorttitle="RSI", format=format.price, precision=2, resolution="")
        len = input(14, minval=1, title="Length")
        src = input(close, "Source", type = input.source)
        up = rma(max(change(src), 0), len)
        down = rma(-min(change(src), 0), len)
        rsi = down == 0 ? 100 : up == 0 ? 0 : 100 - (100 / (1 + up / down))
        plot(rsi, "RSI", color=#8E1599)
        band1 = hline(70, "Upper Band", color=#C0C0C0)
        band0 = hline(30, "Lower Band", color=#C0C0C0)
        fill(band1, band0, color=#9915FF, transp=90, title="Background")

    :param ohlc:
    :param period:
    :param round_rsi:
    :return: an array with the RSI indicator values
    """

    delta = ohlc["closingValues"].diff()

    up = delta.copy()
    up[up < 0] = 0
    up = pd.Series.ewm(up, alpha=1/period).mean()

    down = delta.copy()
    down[down > 0] = 0
    down *= -1
    down = pd.Series.ewm(down, alpha=1/period).mean()

    rsi = np.where(up == 0, 0, np.where(down == 0, 100, 100 - (100 / (1 + up / down))))

    return np.round(rsi, 2) if round_rsi else rsi

In [9]:
def stoch_rsi_tradingview(ohlc: pd.DataFrame, period=14, smoothK=3, smoothD=3):
    """ Calculating Stochastic RSI (gives the same values as TradingView as of March 20, 2021.
        smoothK = input(3, "K", minval=1)
        smoothD = input(3, "D", minval=1)
        lengthRSI = input(14, "RSI Length", minval=1)
        lengthStoch = input(14, "Stochastic Length", minval=1)
        src = input(close, title="RSI Source")
        rsi1 = rsi(src, lengthRSI)
        k = sma(stoch(rsi1, rsi1, rsi1, lengthStoch), smoothK)
        d = sma(k, smoothD)

    :param ohlc:
    :param period:
    :param smoothK:
    :param smoothD:
    :return:
    """
    # Calculate RSI
    rsi = rsi_tradingview(ohlc, period=period, round_rsi=False)

    # Calculate StochRSI
    rsi = pd.Series(rsi)
    stochrsi  = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
    stochrsi_K = stochrsi.rolling(smoothK).mean()
    stochrsi_D = stochrsi_K.rolling(smoothD).mean()
    ohlc['stochrsi_K']=round(stochrsi_K * 100, 2)
    ohlc['stochrsi_D']=round(stochrsi_D * 100, 2)
    

In [10]:
def trader(df: pd.DataFrame,index,symbol,tradeValues):
    print('Here')
    print(tradeValues[symbol])
    if(str(datetime.datetime.fromtimestamp(df['candleStick'][index][0]).time())=='15:15:00'):
        print('StopLoss on '+str(datetime.datetime.fromtimestamp(df['candleStick'][index][0]))+' hit '+str(tradeValues[symbol]['stopLossCounterForDay'])+' times')
        tradeValues[symbol]['stopLossCounterForDay']=0
    #Sells everything if already bought a stock EOD
    if(tradeValues[symbol]['buying']==1):
        if(str(datetime.datetime.fromtimestamp(df['candleStick'][index][0]).time())=='15:15:00'):
            
            print(symbol+' EOD Exit at '+str(df['candleStick'][index][4]))
            tradeValues[symbol]['eodCounter']=tradeValues[symbol]['eodCounter']+((df['candleStick'][index][4]-tradeValues[symbol]['buyPrice'])/(tradeValues[symbol]['buyPrice']-tradeValues[symbol]['stopLoss']))
            tradeValues[symbol]['counter']=0
            tradeValues[symbol]['buyPrice']=-420
            tradeValues[symbol]['stopLoss']=-420
            tradeValues[symbol]['reward']=3
            tradeValues[symbol]['buying']=0
    if(tradeValues[symbol]['stopLossCounterForDay']<2):
        if not math.isnan(df['mean'][index] ):
            print(df['mean'][index])
            print(df['candleStick'][index])
            if not((df['mean'][index]<= df['candleStick'][index][2]) and (df['mean'][index]>= df['candleStick'][index][3])):
                #Checks for reference candles
                if(tradeValues[symbol]['buying']!=1):
                    if(df['mean'][index]>df['candleStick'][index][2] and df['Lower_Band'][index]>=df['candleStick'][index][3]):
                        print(symbol+' Reference Candle '+str(datetime.datetime.fromtimestamp(df['candleStick'][index][0])))
                        tradeValues[symbol]['buyPrice']=df['candleStick'][index][2]
                        tradeValues[symbol]['stopLoss']=df['candleStick'][index][3]-0.2
            else:
                if(tradeValues[symbol]['buyPrice']!=-420 and tradeValues[symbol]['stopLoss']!=-420):
                    #Buy Order
                    targetPrice=tradeValues[symbol]['buyPrice']+((tradeValues[symbol]['buyPrice']-tradeValues[symbol]['stopLoss'])*tradeValues[symbol]['reward'])
                    if(tradeValues[symbol]['buying']==0):
                        print(symbol+' Buy Triggered at '+str(tradeValues[symbol]['buyPrice'])+' '+str(datetime.datetime.fromtimestamp(df['candleStick'][index][0])))
                        tradeValues[symbol]['buying']=1
                        
                    #stoploss hit    
                    elif(df['candleStick'][index][3]<=tradeValues[symbol]['stopLoss']):
                        print(symbol+' SL hit at '+str(tradeValues[symbol]['stopLoss'])+' '+str(datetime.datetime.fromtimestamp(df['candleStick'][index][0])))
                        tradeValues[symbol]['stopLossCounterForDay']=tradeValues[symbol]['stopLossCounterForDay']+1
                        tradeValues[symbol]['buying']=0
                        tradeValues[symbol]['buyPrice']=-420
                        tradeValues[symbol]['stopLoss']=-420
                        tradeValues[symbol]['slCounter']=tradeValues[symbol]['slCounter']+1
                        
                    #Sells if made more then 1:3 and meets stochrsi indicator condition
                    elif (df['candleStick'][index][2]>=targetPrice and df['stochrsi_K'][index]<=df['stochrsi_D'][index]):
                        tradeValues[symbol]['buying']=0
                        print(symbol+' TP hit at '+str(df['candleStick'][index][2])+' '+str(datetime.datetime.fromtimestamp(df['candleStick'][index][0])))
                        #tpCounter=tpCounter+1
                        tradeValues[symbol]['tpCounter']=tradeValues[symbol]['tpCounter']+((df['candleStick'][index][4]-tradeValues[symbol]['buyPrice'])/(tradeValues[symbol]['buyPrice']-tradeValues[symbol]['stopLoss']))
                        tradeValues[symbol]['buyPrice']=-420
                        tradeValues[symbol]['stopLoss']=-420
                    


In [11]:

symbols=["MCX:CRUDEOIL23OCTFUT"]
presentCandleStick={}
candleSticks={}
closingValues={}
valuesForDataframe={}
dataframes={}
candleStickLength=5
lengthOfDF={}
tradeValues={}
for symbol in symbols:
    presentCandleStick[symbol]=[-420]
    candleSticks[symbol]=[]
    closingValues[symbol]=[]
    valuesForDataframe[symbol]={}
    lengthOfDF[symbol]=0
    tradeValues[symbol]={'counter':0,'buyPrice':-420,'stopLoss':-420,'reward':3,'buying':0,'slCounter':0,'tpCounter':0,'eodCounter':0,'stopLossCounterForDay':0}

def run_process_order_update(access_token):
    data_type = "symbolData"
    print(access_token)
    fs = ws.FyersSocket(access_token=access_token,log_path=os.getcwd())
    fs.websocket_data = custom_message
    fs.subscribe(symbol=symbols,data_type=data_type)
    fs.keep_running()


def custom_message(msg): 

    global candleSticks
    global presentCandleStick
    global candleStickLength
    global closingValues
    global valuesForDataframe
    global dataframes
    global tradeValues
    print(msg[0])
    try:
            if(presentCandleStick[msg[0]['symbol']][0]==-420):
                presentCandleStick[msg[0]['symbol']].append(msg[0]['ltp'])
                presentCandleStick[msg[0]['symbol']].append(msg[0]['ltp'])
                presentCandleStick[msg[0]['symbol']].append(msg[0]['ltp'])
                presentCandleStick[msg[0]['symbol']][0]=msg[0]['timestamp']+(candleStickLength*60)
            else:
                if(msg[0]['timestamp']>=presentCandleStick[msg[0]['symbol']][0]):
                    presentCandleStick[msg[0]['symbol']].append(msg[0]['ltp'])
                    if(msg[0]['ltp']>presentCandleStick[msg[0]['symbol']][2]):
                        presentCandleStick[msg[0]['symbol']][2]=msg[0]['ltp']
                    if(msg[0]['ltp']<presentCandleStick[msg[0]['symbol']][3]):
                        presentCandleStick[msg[0]['symbol']][3]=msg[0]['ltp']
                    candleSticks[msg[0]['symbol']].append(presentCandleStick[msg[0]['symbol']])
                    closingValues[msg[0]['symbol']].append(presentCandleStick[msg[0]['symbol']][4])
                    valuesForDataframe[msg[0]['symbol']]['closingValues']=closingValues[msg[0]['symbol']]
                    valuesForDataframe[msg[0]['symbol']]['candleStick']=candleSticks[msg[0]['symbol']]
                    dataframes[msg[0]['symbol']]=pd.DataFrame(valuesForDataframe[msg[0]['symbol']])
                    sma(dataframes[msg[0]['symbol']])
                    stoch_rsi_tradingview(dataframes[msg[0]['symbol']])
                    trader(dataframes[msg[0]['symbol']],(lengthOfDF[msg[0]['symbol']]),msg[0]['symbol'],tradeValues)
                    lengthOfDF[msg[0]['symbol']]=lengthOfDF[msg[0]['symbol']]+1
                    print('CandleStick at '+str(presentCandleStick[msg[0]['symbol']][0])+' for '+msg[0]['symbol']+':'+str(presentCandleStick[msg[0]['symbol']][1])+' '+str(presentCandleStick[msg[0]['symbol']][2])+' '+str(presentCandleStick[msg[0]['symbol']][3])+' '+str(presentCandleStick[msg[0]['symbol']][4]))
                    presentCandleStick[msg[0]['symbol']]=[-420]
                else:
                    if(msg[0]['ltp']>presentCandleStick[msg[0]['symbol']][2]):
                        presentCandleStick[msg[0]['symbol']][2]=msg[0]['ltp']
                    if(msg[0]['ltp']<presentCandleStick[msg[0]['symbol']][3]):
                        presentCandleStick[msg[0]['symbol']][3]=msg[0]['ltp']
    except Exception as e: 
        print(e)
        exit()


def main():
    token_to_use = f"{client_id}:{access_token}"
    run_process_order_update(token_to_use)


if __name__ == '__main__':
  main()






TAZ287TZUA-100:eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE2OTYwNjA3MDUsImV4cCI6MTY5NjEyMDIyNSwibmJmIjoxNjk2MDYwNzA1LCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbEY5VWhhckFEX0hQREh3cU5iOURpVWdDNUNCNm8yWVljVWk5WmwwaVpWNXBLX1lHd1pjU3dfSHZTcWRVNUw0VVk1OE1nNHduN1d0M2FpOUEwUXBVUF9FMzJKQXhYWDVmU3F5ekRVYl8zQm9CNDhubz0iLCJkaXNwbGF5X25hbWUiOiJNT0hBTU1FRCBSQVlZQU4gU0FJVCIsIm9tcyI6IksxIiwiaHNtX2tleSI6IjViNzM4N2U2MThkMzg5ZTRlYzhlZmY1NTgzNGUwZjEwNTY3Y2FlNTg5OWFkOTVmNzM4MTEzN2EzIiwiZnlfaWQiOiJYTTA0MzQyIiwiYXBwVHlwZSI6MTAwLCJwb2FfZmxhZyI6Ik4ifQ.ezffwTYpNMdgqKxkUYDCUgAxIgeCKG0bMrNakUamT8g


: 